In [ ]:
packages <- c("data.table", "ggplot2", "leaflet", "sf", "dplyr")
install_if_missing <- packages[!(packages %in% installed.packages()[,"Package"])]
if(length(install_if_missing)) install.packages(install_if_missing)

library(data.table)
library(leaflet)
library(sf)
library(dplyr)
library(viridis)

In [ ]:
# Load the CSV (auto-mounted in Kaggle)
path = '/kaggle/input/greek-house-market-of-2022-cleaned-with-geo/ghm_2022_cleaned_geo.csv'
df <- fread(path)  

# Quick explore & clean (handle NA, create price_per_sqm)
summary(df)
df <- df[!is.na(res_price) & !is.na(res_sqr) & res_sqr > 0]  # Remove invalid rows
df[, price_per_sqm := res_price / res_sqr]  # New feature

# Filter to valid points (e.g., top 500 for map performance)
# Use 'res_price_sqr' as price_per_sqm (from summary)
map_df <- df[!is.na(lat) & !is.na(long) & !is.na(res_price_sqr)]

In [ ]:
# NUM OF HOUSES PER AREA
print(table(map_df$location_name))  

# LEAFLET

In [ ]:
# 1. Aggregate data by location
map_loc <- map_df %>%
  group_by(location_name, lat, long) %>%
  summarise(
    n_houses = n(),
    avg_price_sqr = mean(res_price_sqr, na.rm = TRUE),
    min_price_sqr = min(res_price_sqr, na.rm = TRUE),
    max_price_sqr = max(res_price_sqr, na.rm = TRUE),
    avg_price = mean(res_price, na.rm = TRUE),
    avg_sqr = mean(res_sqr, na.rm = TRUE),
    .groups = "drop"
  )

# 2. Create darker green-to-red color palette
pal <- colorNumeric(palette = c("#006400", "#FFAA00", "#8B0000"), domain = map_loc$avg_price_sqr)

# 3. Build Leaflet map with small colored circle markers
m <- leaflet(map_loc) %>%
  addTiles() %>%
  addCircleMarkers(
    lng = ~long, lat = ~lat,
    radius = 6,                     
    color = ~pal(avg_price_sqr),    
    fillColor = ~pal(avg_price_sqr),
    fillOpacity = 0.9,
    stroke = TRUE,
    weight = 1,
    popup = ~paste(
      "<b>", location_name, "</b><br>",
      "Houses: ", n_houses, "<br>",
      "Avg Price: €", round(avg_price,0), "<br>",
      "Avg Size: ", round(avg_sqr,1), " sqm<br>",
      "Avg Price/sqm: €", round(avg_price_sqr,2), "<br>",
      "Min Price/sqm: €", round(min_price_sqr,2), "<br>",
      "Max Price/sqm: €", round(max_price_sqr,2)
    )
  ) %>%
  addLegend(
    "bottomright",
    pal = pal,
    values = ~avg_price_sqr,
    title = "Avg Price per sqm (€)",
    opacity = 1
  )

# 4. Display map
m


### Popup Information for a Location

When clicking on a circle on the map (e.g., **Αργυρούπολη**), the popup shows:

- **Location Name:** Αργυρούπολη  
- **Houses:** 211  
- **Average Price:** €318,227  
- **Average Size:** 103.7 sqm  
- **Average Price per sqm:** €3,026.71  
- **Minimum Price per sqm:** €1,077  
- **Maximum Price per sqm:** €5,360  

This provides a quick overview of the real estate market in the area, showing the typical price, size, and price per square meter range for properties in that location.
